In [62]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option("max_rows", None)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import glob


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
!/anaconda/envs/azureml_py38_pytorch/bin/python predict.py --input_path="../HKTVMall_data" --input_csv="../human_detection/result/frcnn_result.csv" --face_path="detect_faces" --save_path="result/face_attributes.csv" --model_path="resnet_model/fairface_4race_20191111.pt" --face_size=500 --progress=20

using CUDA: True
detecting faces, saved at result/face_attributes.csv
---0/207---
Sorry, there were no faces found in '../HKTVMall_data/3.jpg'
Sorry, there were no faces found in '../HKTVMall_data/6.jpg'
Sorry, there were no faces found in '../HKTVMall_data/7.jpg'
Sorry, there were no faces found in '../HKTVMall_data/40.jpg'
Sorry, there were no faces found in '../HKTVMall_data/46.jpg'
Sorry, there were no faces found in '../HKTVMall_data/51.jpg'
Sorry, there were no faces found in '../HKTVMall_data/53.jpg'
Sorry, there were no faces found in '../HKTVMall_data/56.jpg'
Sorry, there were no faces found in '../HKTVMall_data/74.jpg'
Sorry, there were no faces found in '../HKTVMall_data/83.jpg'
---20/207---
Sorry, there were no faces found in '../HKTVMall_data/87.jpg'
Sorry, there were no faces found in '../HKTVMall_data/92.jpg'
Sorry, there were no faces found in '../HKTVMall_data/96.jpg'
Sorry, there were no faces found in '../HKTVMall_data/112.jpg'
Sorry, there were no faces found in '..

In [68]:
def evaluate(df, feat_name, pred_col, label_col):
    print(feat_name, " - accuracy: %.2f" % (len(df[df[pred_col] == res[label_col]]) / len(df)))

In [86]:
pred = pd.read_csv("result/face_attributes.csv")
label = pd.read_csv("../human_detection/result/true_label.csv")

detect_face_cnt = len(pred["image_id"].unique())
face_cnt = len(pred)

# filter true label - detect face
res = label[label["gender"].notnull()].merge(
    pred, on="image_id", how="left", suffixes=["_true", "_pred"]
    )

image_paths = glob.glob("../HKTVMall_data/*jpg")
n_images = len(image_paths)

df = pd.read_csv("../human_detection/result/frcnn_result.csv")
detect_human_cnt = df["pred_label"].sum()

face_detect_acc = len(res[res["face_idx"].notnull()]) / len(res)

print("Test on %d images." % n_images)
print("%d images contain human." % detect_human_cnt)
print("%d images have human faces. Detect %d faces. accuracy - %.2f" % (detect_face_cnt, face_cnt, face_detect_acc))
evaluate(res, "Race", "race_pred", "race_true")
evaluate(res, "Gender", "gender_pred", "gender_true")
evaluate(res, "Age group", "age_group_pred", "age_group_true")

Test on 463 images.
207 images contain human.
88 images have human faces. Detect 103 faces. accuracy - 0.86
Race  - accuracy: 0.72
Gender  - accuracy: 0.66
Age group  - accuracy: 0.00
